In [4]:
import pandas as pd
import numpy as np
import time
import re
import csv
import math
from tqdm.notebook import tqdm
tqdm.pandas()

from exact import solve
from automaton import evaluate
from util import lane_order

Solving with each setting of cutting planes.

In [11]:
def eval_exact(data):
    def solve_instance(x, cut):
        start = time.time()
        y = solve(x['instance'], cutting_planes=cut)
        return y['y'], y['obj'], time.time() - start

    for cut in [0,1,2]:
        # apply branch-and-bound with different cutting planes active
        data[[f'opt_y_{cut}', f'opt_obj_{cut}', f'running_time_{cut}']] = \
            data.progress_apply(lambda x: solve_instance(x, cut), axis=1, result_type='expand')

    # we verify the cutting planes by comparing objectives
    data['matching1'] = data.apply(
        lambda x: math.isclose(x['opt_obj_0'], x['opt_obj_1'], rel_tol=1e-04), axis=1)
    data['matching2'] = data.apply(
        lambda x: math.isclose(x['opt_obj_0'], x['opt_obj_2'], rel_tol=1e-04), axis=1)
    assert data['matching1'].all() and data['matching2'].all()

    # rename and remove columns
    data['opt_y'] = data['opt_y_1']
    data['opt_obj'] = data['opt_obj_1']
    data = data.drop([
            'opt_y_0', 'opt_obj_0',
            'opt_y_1', 'opt_obj_1',
            'opt_y_2', 'opt_obj_2',
            'matching1', 'matching2'
       ], axis=1, errors='ignore')

    # store schedules with their lane order
    data['opt_eta'] = data.apply(lambda x: lane_order(x['opt_y']), axis=1)

    return data

Solve optimally for the given instances.

In [12]:
sets = [1,2,3,4,5,6]
files = [f"data/test_{i}.pkl" for i in sets]

for file in files:
    print(f"processing {file}")
    data = pd.read_pickle(file)

    print('branch-and-bound')
    data = eval_exact(data)

    # save everything with pickle
    data.to_pickle(file)

processing data/test_5.pkl
branch-and-bound


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

processing data/test_6.pkl
branch-and-bound


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Save the results to `running_times.csv` for report.

In [13]:
total = pd.DataFrame()
for test_file in files:
    i = int(re.findall(r'\d+', test_file)[0])
    data_set = pd.read_pickle(test_file)
    data_set['set_id'] = i
    total = pd.concat([total, data_set])


total = total[['set_id', 'opt_obj', 'threshold_obj', 'neural_obj']]
print(total)
#total.to_csv('../report/data/results.csv', index=False)

                                             instance  threshold_obj  \
0   {'release': [[2.1825253396400552, 9.8624692089...     373.726948   
1   {'release': [[1.3741147147927595, 4.8811769587...     349.482294   
2   {'release': [[2.9523270629583163, 6.0075320385...     341.560127   
3   {'release': [[4.3732550121114455, 7.3339977972...     344.306399   
4   {'release': [[1.0285181116826017, 3.5748770271...     312.384719   
..                                                ...            ...   
95  {'release': [[10.50897870604066, 12.0523213983...    1031.480453   
96  {'release': [[5.637586675461332, 10.4871323435...     938.006767   
97  {'release': [[5.619585354306989, 9.33486682095...     872.102500   
98  {'release': [[0.49492085463299007, 6.048997660...     804.278330   
99  {'release': [[3.0317342218802037, 4.4865882437...     779.789558   

    neural_obj  running_time_1  \
0   357.923812        0.039579   
1   340.559329        0.065648   
2   329.047824        0.035969   